In [3]:
!pip install deep_translator
!pip install vaderSentiment
!pip install wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 7.5 MB/s eta 0:00:00


In [19]:
import pandas as pd
from deep_translator import GoogleTranslator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [5]:
from google.colab import files
uploaded = files.upload()

Saving IA.parquet to IA.parquet


In [6]:
df = pd.read_parquet('IA.parquet', engine='pyarrow')
df.head()

,store,assigned_at,answer
0,None,2023-02-21 07:18:07.598950,comentario
1,FN60,2023-02-20 12:25:30.652022,"Todo lo hacen perfecto,, gracias por su atención"
2,T002,2023-02-18 14:10:31.610430,Offcorss...siempre me ha parecido una buena al...
3,T013,2023-02-23 12:40:51.640983,Me encanta las tiendas el servicio y sus produ...
4,FN54,2023-02-20 17:10:31.230531,Ya no ay probadores


In [7]:
print(df.describe())

                         assigned_at
count                          17050
mean   2023-01-03 05:23:45.393767424
min       2022-01-02 11:45:56.643000
25%    2022-06-11 12:10:30.318237184
50%    2022-11-27 20:18:07.706745856
75%    2023-08-19 15:55:16.828758016
max       2024-07-27 13:55:21.613611


In [ ]:
# verificar nulos
print(df.isnull().sum())

store           75
assigned_at      0
answer         987
dtype: int64


In [8]:
# Eliminar nulos
df.dropna(inplace=True)
print(df.isnull().sum())

store          0
assigned_at    0
answer         0
dtype: int64


In [22]:
# Filtrar valores no textuales y garantizar que solo se traduzcan cadenas de texto
df['answer'] = df['answer'].apply(lambda x: x if isinstance(x, str) else '')

In [28]:
import re

In [29]:
# Eliminar signos de puntuación
df['answer'] = df['answer'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['answer'].head()

,answer
1,Todo lo hacen perfecto gracias por su atención
2,Offcorsssiempre me ha parecido una buena alter...
3,Me encanta las tiendas el servicio y sus produ...
4,Ya no ay probadores
5,Excelente servicio


In [9]:
# Inicializar el traductor y el analizador de sentimientos
translator = GoogleTranslator(source='es', target='en')
sia = SentimentIntensityAnalyzer()

In [32]:
import pandas as pd
from textblob import TextBlob

In [ ]:
# Función para corregir la ortografía
def corregir_ortografia(texto):
    if isinstance(texto, str) and len(texto) > 0:
        try:
            blob = TextBlob(texto)
            return str(blob.correct())  # Corregir ortografía y devolver como string
        except Exception as e:
            print(f"Error during spell correction: {e}")
            return texto  # Devolver el texto original si hay un error
    else:
        return texto  # Si no es texto válido, devolver el original

# Aplicar la corrección de ortografía a la columna 'answer'
df['answer_corrected'] = df['answer'].apply(corregir_ortografia)

# Mostrar las correcciones
print(df[['answer', 'answer_corrected']])


In [10]:
# Función para analizar sentimiento
def analizar_sentimiento(texto):
    # Check if text is not None and not empty
    if texto and isinstance(texto, str):
        # Traducir el texto a inglés
        text_en = translator.translate(texto)
        # Check if translation is successful
        if text_en:
            # Analizar el sentimiento
            sentiment = sia.polarity_scores(text_en)
            return sentiment
        else:
            # Return default sentiment if translation fails
            return {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
    else:
        # Return default sentiment if text is None or empty
        return {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}

In [11]:
# Aplicar el análisis de sentimiento
df['answer_trs'] = df['answer'].apply(analizar_sentimiento)

TooManyRequests: Server Error: You made too many requests to the server.According to google, you are allowed to make 5 requests per secondand up to 200k requests per day. You can wait and try again later oryou can try the translate_batch function

In [15]:
import nltk # import nltk library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon') # download vader lexicon

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [20]:
from deep_translator.exceptions import TooManyRequests

In [30]:
!pip install deep-translator
import time
from deep_translator import GoogleTranslator
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# ... (your existing code for loading data and preprocessing) ...

# Inicializar el traductor y el analizador de sentimientos
translator = GoogleTranslator(source='es', target='en')
sia = SentimentIntensityAnalyzer()

# Función para analizar sentimiento con manejo de errores y retardo
def analizar_sentimiento(texto):
    # Check if text is not None and not empty
    if texto and isinstance(texto, str) and len(texto) > 0:
        try:
            # Traducir el texto a inglés con retardo
            text_en = translator.translate(texto)
            time.sleep(0.1)  # Add a delay of 0.2 seconds between requests
            # Analizar el sentimiento
            sentiment = sia.polarity_scores(text_en)
            return sentiment
        except TooManyRequests:
            print("Too many requests. Waiting for 60 seconds...")
            time.sleep(0.2)  # Wait for 60 seconds before retrying
            return analizar_sentimiento(texto)  # Retry translation
        except Exception as e:
            print(f"Error during translation: {e}")
            return {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}  # Return default sentiment if error
    else:
        # Return default sentiment if text is None or empty
        return {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}

# Aplicar el análisis de sentimiento
df['answer_trs'] = df['answer'].apply(analizar_sentimiento)

KeyboardInterrupt: 